<a href="https://colab.research.google.com/github/andresum97/03MAIR_algoritmos_de_optimizacion/blob/main/Actividad_Guiada_3_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG3 - Actividad Guiada 3

Nombre: Andrés Urízar

https://colab.research.google.com/drive/1Sylfi3efLc615xLhlNuGVtc01hfVNyxm?usp=sharing

https://github.com/andresum97/03MAIR_algoritmos_de_optimizacion


## Carga de librerias


In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Generar valores aleatorios

In [3]:
#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos 

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [4]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file) #Datos del fichero

#Nodos
nodos = list(problem.get_nodes()) #Tiene funcion definida para obtener nodos

#Aristas
Aristas = list(problem.get_edges()) #Tiene funcion definida para obtener aristas

In [5]:
# Funcion para probar

#Distancia entre nodos
problem.get_weight(0, 3)

23

## Declaración de funciones básicas

In [41]:
# Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.get_weight(a,b)

# Genera una solucion aleatoria empezando en el nodo 0
def crear_solucion(nodos):
    solucion = [nodos[0]]
    #en cada paso se elige un elemento no elegido y eliminando el 0 y los que
    #hayamos elegido anteriormente
    #Si hay que quitar el nodo 0 porque si parece afectar la distancia
    for nodo in nodos[1:]:
      solucion += [random.choice(list(set(nodos) - set({nodos[0]}) - set(solucion)))]
    return solucion

# Calcula la distancia total de la trayectoria/solucion
def distancia_total(solucion, problem):
  dist = 0
  #recorrer todos los pasos y calcula distancia acumulando las aristas
  for i in range(len(solucion)-1):
    dist += distancia(solucion[i], solucion[i+1], problem)
  return dist + distancia(solucion[len(solucion)-1], solucion[0], problem)

solucion = crear_solucion(nodos)
distancia_total(solucion, problem)


4680

## Búsqueda aleatoria

In [42]:
# Busqueda aleatoria

def busqueda_aleatoria(problem, n):
  nodos = list(problem.get_nodes())

  mejor_resultado = []
  #Se comienza con un valor alto para que el primer resultado encontrado
  #lo reemplace
  mejor_distancia = 999999999

  for i in range(n):
    resultado = crear_solucion(nodos)    #solucion aleatoria
    distancia = distancia_total(solucion, problem) #distancia de solucion aleatoria

    if distancia < mejor_distancia:
      mejor_resultado = resultado
      mejor_distancia = distancia

  return mejor_resultado, distancia

solucion, dist = busqueda_aleatoria(problem, 5000)
print(solucion)
print(dist)

[0, 18, 21, 38, 11, 9, 12, 14, 5, 19, 37, 24, 36, 16, 29, 31, 34, 6, 25, 35, 13, 27, 30, 40, 3, 20, 26, 32, 2, 28, 33, 4, 22, 1, 8, 10, 17, 39, 23, 41, 15, 7]
4680


## Búsqueda local

Te quedaste en el minuto 51:58

In [35]:
# Busqueda local
def genera_vecina(solucion):
  mejor_resultado = []
  mejor_distancia = 9999999999

  # Se intercambian nodos, se comienza por el primer nodo
  # y se utiliza el nodo siguiente al seleccionado hasta el final
  for i in range(1, len(solucion) - 1):
    for j in range(i+1, len(solucion)):
      # Se procede a intercambiar nodo i con nodo j
      vecino = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      # Calcula la nueva distancia de la solucion obtenida
      nueva_distancia = distancia_total(vecino, problem)

      # Si es mejor se realiza el intercambio
      if nueva_distancia <= mejor_distancia:
        mejor_distancia = nueva_distancia
        mejor_resultado = vecino

  return mejor_resultado

print(distancia_total(solucion, problem))
nueva_solucion = genera_vecina(solucion)
print(distancia_total(nueva_solucion, problem))

4889
4284


In [49]:
#Busqueda local

def busqueda_local(problem):
  mejor_resultado = []

  # Se genra una solucion aleatoria
  sol_referencia = crear_solucion(nodos)
  mejor_distancia = distancia_total(sol_referencia, problem)

  cont = 0
  # Se realiza un loop infinito hasta que se haga un retorno
  # (criterio de parada)
  while(True):
    vecino = genera_vecina(sol_referencia)
    distancia = distancia_total(vecino, problem)

    # Se comprueba que mejora el resultado
    if distancia < mejor_distancia:
      mejor_resultado = vecino
      mejor_distancia = distancia
    
    else:
      # Cuando ya no tiene a donde mas avanzar
      print('Distancia', mejor_distancia)
      return mejor_resultado

    sol_referencia = vecino


resultado = busqueda_local(problem)
print('Distancia ', resultado)

Distancia 1785
Distancia  [0, 6, 5, 26, 18, 12, 10, 8, 29, 30, 28, 27, 2, 4, 13, 19, 14, 16, 15, 37, 20, 33, 34, 3, 1, 7, 31, 32, 38, 22, 39, 24, 40, 21, 9, 23, 41, 25, 11, 17, 36, 35]


## Recocido simulado

In [55]:
# Recocido simulado o simulated annealing

# Es para generar aleatoriamente un x en el entorno de vecindad de la
# la solucion de referencia
# No se debe elegir tan aleatorio, sino algo random pero mas dirigido
# considerar eso para nodo a,b. Verificar quiza nodos alejados.
def genera_vecina_aleatorio(solucion):
  #Seleccion de nodos de manera aleatoria
  a,b = sorted(random.sample(range(1, len(solucion)), 2))

  return solucion[:a] + [solucion[b]] + solucion[a+1:b] + [solucion[a]] + solucion[b+1:]

In [51]:
# Maneja probabilidad de transparencia para aceptar peores soluciones
def probabilidad(temperatura, delta):
  return random.random() < math.exp(-1*delta / temperatura)

# Maneja el descenso de temperatura
def bajar_temperatura(temperatura):
  return temperatura*0.99

In [58]:
def recocido_simulado(problem, temperatura):
  sol_referencia = crear_solucion(nodos)
  distancia_referencia = distancia_total(sol_referencia, problem)

  solucion = []    # x* del algoritmo
  distancia = 9999999999   # F* del algoritmo

  n = 0
  while temperatura > 0.001:
    n += 1
    # Aqui se desarrolla la propuesta de mejora
    vecino = genera_vecina_aleatorio(sol_referencia)
    distancia_vecino = distancia_total(vecino, problem)

    # Si es mejor hace el cambio
    if distancia_vecino < distancia:
      solucion = vecino
      distancia = distancia_vecino

    # En caso no sea mejor, se realiza el salto por medio
    # de la temperatura, cambiando una solucion diferente
    if (
        distancia_vecino < distancia_referencia 
        or probabilidad(
            temperatura, 
            abs(distancia_referencia - distancia_vecino)
            )
        ):
      sol_referencia = vecino
      distancia_referencia = distancia_vecino

    # Se baja la temperatura
    temperatura = bajar_temperatura(temperatura)

  print("mejor distancia ", distancia)
  return solucion


sol = recocido_simulado(problem, 10000000)
print("Solucion ", sol)
    

mejor distancia  1811
Solucion  [0, 1, 32, 31, 7, 17, 37, 14, 26, 10, 8, 23, 40, 9, 41, 25, 11, 12, 18, 13, 16, 15, 19, 5, 6, 4, 3, 27, 2, 28, 30, 29, 39, 21, 24, 22, 38, 33, 34, 20, 36, 35]
